# Reading Benchmark

## Pandas - parquet.gzip format

In [1]:
import pyarrow
import pandas as pd

/Users/peter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
%%time
df = pd.read_parquet("./data/pdb2uniprot_residues.parquet.gzip", engine='pyarrow')
print("Number of rows: " + str(df.shape[0]))

Number of rows: 105594971
CPU times: user 1min 1s, sys: 21.8 s, total: 1min 23s
Wall time: 1min 26s


In [3]:
df.head()

,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum
0,5HZA.B,454,231,Q5F4T5,454
1,5HZA.B,455,232,Q5F4T5,455
2,5HZA.B,456,233,Q5F4T5,456
3,5HZA.B,457,234,Q5F4T5,457
4,5HZA.B,458,235,Q5F4T5,458


## PySpark - parquet.gzip format

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").appName("Benchmark").getOrCreate()

In [6]:
%%time
ds = spark.read.parquet("./data/pdb2uniprot_residues.parquet.gzip").cache()
print("Number of rows: " + str(ds.count()))

Number of rows: 105594971
CPU times: user 17.2 ms, sys: 12.9 ms, total: 30.1 ms
Wall time: 2min 54s


In [7]:
ds.show(5)

+----------------+---------+---------+---------+----------+
|structureChainId|pdbResNum|pdbSeqNum|uniprotId|uniprotNum|
+----------------+---------+---------+---------+----------+
|          5HZA.B|      454|      231|   Q5F4T5|       454|
|          5HZA.B|      455|      232|   Q5F4T5|       455|
|          5HZA.B|      456|      233|   Q5F4T5|       456|
|          5HZA.B|      457|      234|   Q5F4T5|       457|
|          5HZA.B|      458|      235|   Q5F4T5|       458|
+----------------+---------+---------+---------+----------+
only showing top 5 rows



In [8]:
spark.stop()

## PySpark - orc.lzo format

In [9]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.master("local[*]").appName("Benchmark").getOrCreate()
spark.conf.set("spark.sql.orc.impl", "native")
spark.conf.set("spark.sql.orc.enableVectorizedReader", True)

In [11]:
%%time
ds = spark.read.orc("./data/pdb2uniprot_residues.orc.lzo").cache()
print("Number of rows: " + str(ds.count()))

Number of rows: 105594971
CPU times: user 9.68 ms, sys: 6.99 ms, total: 16.7 ms
Wall time: 1min 25s


In [12]:
ds.show(5)

+----------------+---------+---------+---------+----------+
|structureChainId|pdbResNum|pdbSeqNum|uniprotId|uniprotNum|
+----------------+---------+---------+---------+----------+
|          5HZA.B|      454|      231|   Q5F4T5|       454|
|          5HZA.B|      455|      232|   Q5F4T5|       455|
|          5HZA.B|      456|      233|   Q5F4T5|       456|
|          5HZA.B|      457|      234|   Q5F4T5|       457|
|          5HZA.B|      458|      235|   Q5F4T5|       458|
+----------------+---------+---------+---------+----------+
only showing top 5 rows



In [13]:
spark.stop()